In [32]:
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm

# client = MongoClient("mongodb://10.0.52.71:27017/")
# db = client["colabfit-2023-5-16"]

client = MongoClient("mongodb://localhost:5000/")
db = client["colabfit-2023-5-16"]

# client = MongoClient("mongodb://localhost:27017/")
# db = client["mp"]

In [28]:
ds = "DS_sbwtdhegp9yg_0"  # 3BPA
# ds = "DS_8cge9vvgyk47_0" # Local Materials Project

In [50]:
def get_pi_do_md(ds_id):
    pi_ids = defaultdict(list)
    dos = db.data_objects.find({"relationships.datasets": ds_id}, {"colabfit-id": 1})
    for do in tqdm(dos):
        pis = db.property_instances.find(
            {"relationships.data_objects": do["colabfit-id"]},
            {
                "colabfit-id": 1,
                "relationships.data_objects": 1,
                "relationships.metadata": 1,
            },
        )

        for pi in pis:
            pi_ids[pi["colabfit-id"]] = (
                pi["relationships"]["data_objects"],
                pi["relationships"]["metadata"],
            )
    return pi_ids

In [78]:
def get_pi_data(ds_id):
    name = db.datasets.find_one({"colabfit-id": ds_id}, {"name": 1})
    name = name["name"]
    pis = get_pi_do_md(ds_id)
    mds_over_one = sum([len(val[1]) > 1 for k, val in pis.items()])
    unequal_md_do = sum(
        [(len(val[1]) > 1 and len(val[0]) != len(val[1])) for k, val in pis.items()]
    )

    with open(f"{name}_pi_data.txt", "w") as f:
        f.write(f"{name}, {ds_id}\n")
        f.write(f"mds_over_one, {mds_over_one}\n")
        f.write(f"unequal_md_do, {unequal_md_do}\n")
        f.writelines([f"{key}, {val}\n" for key, val in pis.items()])
    print(name)
    return pis

In [79]:
pis = get_pi_data("DS_p6m0q7c6dd64_0")

1691it [00:07, 236.87it/s]

AgPd_npj2021


In [81]:
dss = sorted([ds["colabfit-id"] for ds in db.datasets.find({}, {"colabfit-id": 1})])

3BPA: all 1:1 DO-MD relationships


makelist unequal_md_do_relationships
for each data object
    get relevant PIs
    if relationship is 1(MD)->1(DO):
        find DO-id position in array
        find corresponding MD in array
        DO.update with field "relationships.metadata(array)"
    elif relationship is 1(MD)->many(DO):
        DO.update with field "relationships.metadata(array)"
    else:
        append DO-id to unequal_md_do_relationships (to examine later)

    

db.property_instances.find(
    {
        "$and": [
            {"relationships.data_objects.1": {"$exists": true}},
            {"relationships.metadata.1": {"$exists": true}},
            {
                {"$size": "relationships.metadata"}: {
                    "$not": {"$size": "relationships.data_objects"}
                }
            },
        ]
    }
).count()
{{"$size": "relationships.metadata"}: {"$not": {"$size": "relationships.data_objects"}}}

db.property_instances.aggregate([
  {
    $match:
      {
        "relationships.metadata.1": {
          $exists: true,
        },
        "relationships.data_objects.1": {
          $exists: true,
        },
      },
  },
  {
    $project:
      {
        mds: {
          $size: "$relationships.metadata",
        },
        dos: {
          $size: "$relationships.data_objects",
        },
      },
  },
  {
    $match:
      {
        mds: {
          $ne: "dos",
        },
      },
  },
  {
    $count:
      "num_mds_ne_dos",
  },
])